### Identifying RRL stars in the PAL5 globular cluster

This notebook aims to find any RR Lyrae stars within the Palomar 5 globular cluster and construct light curves of them. This program starts by performing PSF photometry on the 12 time epochs, complete with uncertainties and corrections, for both the 3.6um and 4.5um channels. Once the photometry has been completed, each epoch's stars will be matched with the either the previous epoch or epoch 01 using dummy variables and saved to a new file. These files will then be read in and the magnitudes for each relevant star(s) extracted and plotted against the time epochs to produce the lightcurves.

Attempts will be made to introduce the functions and automation but may require significant manual editing of parameters for each run through.

Contents: 1) list of imports

2) PSF Photometry: -read in epoch01 and build ePSF model -perform PSF photometry using ePSF model for all epochs within a glob.glob loop, saving the files in the format: PAL5_PSFphot_01_epochxx_channelxpxum.fits

3) Star Matching: using WCS match stars in each epoch with either the epoch before or epoch 01 and save to file to avoid duplicating the code 6+ times (use dummy variables and iterate over file number?), saved as: PAL5_01_epochxx_channel_xpxum_matched.fits

4) Extracting magnitudes: reload in the files and extract the magnitudes and uncertainties for stars of interest using the catalog at: http://www.astro.utoronto.ca/~cclement/cat/C1513p000

5) Lightcurves: plot the lightcurves

In [1]:
import math
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy import wcs
from astropy import units as u
from astropy.io import fits # used for FITS file management
from astropy.io import ascii
from astropy.time import Time
from astropy.stats import sigma_clipped_stats # used within star detection
from astropy.table import Table
from astropy.nddata import NDData
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import Angle
from astropy.visualization import simple_norm
from astropy.modeling.fitting import LevMarLSQFitter
from photutils import aperture_photometry # used to perform photometry using annuli
from photutils import DAOStarFinder # used for the star finding algorithm
from photutils import CircularAperture, CircularAnnulus
from photutils import EPSFBuilder
from photutils.psf import extract_stars
from photutils.psf import DAOGroup
from photutils.psf import IterativelySubtractedPSFPhotometry
from photutils.background import MMMBackground
from matplotlib.colors import LogNorm

In [ ]:
## BASE DATA DIRECTORY ##

base_dir = 'PAL5_data/*/'
general_dir = 'PAL5_data/'
channel = '3p6um'

## CHANNEL ##

if channel == '3p6um':
    aper_corr = 1.1233         # aperture correction for 337 (6,6,14) apertures in channel 1, given in IRAC handbook §4.10
    ap_err = aper_corr * 0.02  # uncertainty to ~2% as per IRAC handbook
    zmag = 18.80               # zeropoint magnitude given in IRAC handbook §4.8
    zmag_err = 0.02            # uncertainty calculated from F0 = 280.9 +/- 4.1 in IRAC handbook
elif channel == '4p5um':
    aper_corr = 1.1336
    ap_err = aper_corr * 0.02
    zmag = 18.32
    zmag_err = 0.02 
    
#____________________________________#
## BUILD ePSF MODEL USING ONE IMAGE ##

epoch_dir = 'PAL5_data/PAL5__e1/'

epsf_file = epoch_dir+'PAL5__e1_'+channel+'.fits'

## OPENING FITS FILE AND EXTRACTING DATA ##

with fits.open(epsf_file) as header_list:
    header = header_list[0].header
    fluxconv = header['FLUXCONV']
    exptime = header['EXPTIME']
    time = Time(header['DATE_OBS'])
    counts = exptime / fluxconv
    image_data = fits.getdata(epsf_file, ext = 0)
    data = image_data * counts
    print('FITS file information:\nFILE = {0}\nDATE = {1}\nFLUXCONV = {2}\nEXPTIME = {3}\n\n'.format(epsf_file, time, fluxconv, exptime))

## PARAMETERS : ePSF ##

fwhm = 5.
sigma_val = 6.
model_threshold = 100.
roundlo = -0.5
roundhi = 0.5
sharphi = 0.9
do_plot = True

mean, median, std = sigma_clipped_stats(data, sigma = sigma_val) # sigma-clipping on data

starfind_init = DAOStarFinder(fwhm = fwhm, threshold = model_threshold * std, roundlo = roundlo, roundhi = roundhi, sharphi = sharphi)
epsf_sources = starfind_init(data)
print('Number of ePSF sources found = {0}\n'.format(len(epsf_sources)))

if do_plot == True:
    # PLOT SELECTED STARS TO TEST PARAMETERS #
    positions = np.transpose((epsf_sources['xcentroid'], epsf_sources['ycentroid']))
    apertures = CircularAperture(positions, r = 6.)

    plt.imshow(data, cmap = 'viridis', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')
    apertures.plot(color = 'black', lw = 1.)
    plt.colorbar(fraction = 0.05)
    plt.title('Selected ePSF model stars with params: threshold {0} * std, fwhm = {1}, round = ±{2}, sharphi = {3}'
              .format(model_threshold, fwhm, roundhi, sharphi))
    plt.grid(b = True, which = 'major', lw = .5, color = 'black')
    plt.grid(b = True, which = 'minor', lw = .5, color = 'black')
    plt.gcf().set_size_inches(15, 8)
    plt.show()
    plt.close()

elif do_plot == False:
    pass

## STAR CUTOUTS FOR ePSF ##

cutout_size = 200
hsize = (cutout_size - 1) / 2
x = epsf_sources['xcentroid']
y = epsf_sources['ycentroid']
mask = ((x > hsize) & (x < (data.shape[1] - 1 - hsize)) &
       (y > hsize) & (y < (data.shape[0] - 1 - hsize)))

star_tbl = Table()      # build table of star sources
star_tbl['x'] = x[mask]
star_tbl['y'] = y[mask]
star_tbl['id'] = range(len(star_tbl))
print('Number of refined ePSF sources = {0}\n'.format(len(star_tbl)))

if do_plot == True:
    # PLOT CUTOUT STARS TO BE USED IN EPSF MODEL #
    cutout_pos = np.transpose((star_tbl['x'], star_tbl['y']))
    cutout_apers = CircularAperture(cutout_pos, r = 6.)

    plt.imshow(data, cmap = 'viridis', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')
    cutout_apers.plot(color = 'black', lw = 1.)
    plt.colorbar(fraction = 0.05)
    plt.title('Selected cutout ePSF model stars with params: threshold {0} * std, fwhm = {1}, round = ±{2}, sharphi = {3}'
              .format(model_threshold, fwhm, roundhi, sharphi))
    plt.grid(b = True, which = 'major', lw = .5, color = 'black')
    plt.grid(b = True, which = 'minor', lw = .5, color = 'black')
    plt.gcf().set_size_inches(15, 8)
    plt.show()
    
elif do_plot == False:
    pass

## EXTRACT STARS ##

mean_val, median_val, std_val = sigma_clipped_stats(data, sigma = sigma_val)
epsf_data = data - median_val

while True:
    nddata = NDData(data = epsf_data)
    epsf_stars = extract_stars(nddata, star_tbl, size = 25)

    ## VISUALISE MODEL STARS ##
    ncols = 6
    nrows = int(np.floor(len(epsf_stars) / ncols))
    remaining = len(epsf_stars) - ncols * nrows

    fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (20,20), squeeze = True)
    ax = ax.ravel()
    for i in range(nrows * ncols):
        norm = simple_norm(epsf_stars[i], 'log', percent = 99.)
        ax[i].imshow(epsf_stars[i], cmap = 'viridis', norm = norm, origin = 'lower')
        ax[i].set_title([i])
    plt.show()
    for i in range(ncols * nrows, len(epsf_stars)):
        norm = simple_norm(epsf_stars[i], 'log', percent = 99.)
        ax[i - ncols * nrows].imshow(epsf_stars[i], cmap = 'viridis', norm = norm, origin = 'lower')
        ax[i - ncols * nrows].set_title([i])
    plt.show()
    plt.close()
    
    ## BUILD ePSF ##

    epsf_builder = EPSFBuilder(oversampling = 2, maxiters = 10, progress_bar = True)
    epsf, fitter = epsf_builder(epsf_stars)

    norm = simple_norm(epsf.data, 'log', percent = 99.)
    plt.imshow(epsf.data, cmap = 'viridis', norm = norm, origin = 'lower')
    plt.title('constructed ePSF model')
    plt.colorbar()
    plt.show()
    
    print('Remove any poor quality model stars from the model by their ID. Use ID -99 to escape')
    remove = [int(k) for k in input().split()]
    
    if remove == [-99]:
        print('Escape completed\n')
        break
    
    star_tbl.add_index('id')
    bad_id = star_tbl.loc_indices[remove]
    star_tbl.remove_rows(bad_id)

In [ ]:
## PSF PHOTOMETRY ON ALL EPOCHS ##

## PARAMETERS ##

sigma_psf = 4.
fwhm = 5.
r_ap = 6.
r_in = 6.
r_out = 14.
roundlo = -0.5
roundhi = 0.5
sharphi = 1.0
do_plot = False

## EPOCH LOOP COUNTER ##

epoch = 0

## PSF PHOTOMETRY LOOP ##
for file in glob.glob(base_dir+'PAL5__e[0-9]_'+channel+'.fits', recursive = True) + glob.glob(base_dir+'PAL5__e[0-9][0-9]_'+channel+'.fits', recursive = True):
    epoch += 1
    print('EPOCH NUMBER = {0}\n'.format(epoch))
    ## OPENING FITS FILE AND EXTRACTING DATA ##
    with fits.open(file) as header_list:
        header = header_list[0].header
        fluxconv = header['FLUXCONV']
        exptime = header['EXPTIME']
        time = Time(header['DATE_OBS'])
        counts = exptime / fluxconv
        
        image_data = fits.getdata(file, ext = 0)
        data = image_data * counts
        
        print('FITS file information:\nFILE = {0}\nDATE = {1}\nFLUXCONV = {2}\nEXPTIME = {3}\n\n'.format(file, time, fluxconv, exptime))

    ## EXTRACTING LOC-DEPENDENT CORRECTIONS ##
    corr_file = general_dir+'PAL5__e'+str(epoch)+'/PAL5__e'+str(epoch)+'_correction_'+channel+'.fits'
    with fits.open(corr_file) as hdu_list:
        corr_data = hdu_list[0].data
    
    ## SOURCE DETECTION ON IMAGE ## 
    psf_daofind = DAOStarFinder(fwhm = fwhm, threshold = sigma_psf * std, roundlo = roundlo, roundhi = roundhi)
    psf_sources = psf_daofind(data)

    psf_positions = np.transpose((psf_sources['xcentroid'], psf_sources['ycentroid']))
    psf_apertures = CircularAperture(psf_positions, r = 6.)

    plt.imshow(data, cmap = 'viridis', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')
    psf_apertures.plot(color = 'black', lw = 1.)
    plt.colorbar(fraction = 0.05)
    plt.title('Detected PSF stars: threshold {0} * std, fwhm = {1}, round = ±{2}, sharphi = {3}'
              .format(sigma_psf, fwhm, roundhi, sharphi))
    plt.grid(b = True, which = 'major', lw = .5, color = 'black')
    plt.grid(b = True, which = 'minor', lw = .5, color = 'black')
    plt.gcf().set_size_inches(15, 8)
    plt.show()
    plt.close()
    print('Number of stars detected = {0}\n'.format(len(psf_sources)))
    
    ## GROUP ##

    psf_sources['xcentroid'].name = 'x_0'
    psf_sources['ycentroid'].name = 'y_0'

    daogroup = DAOGroup(crit_separation = sigma_psf * fwhm)
    bkg_estimator = MMMBackground()
    fitter = LevMarLSQFitter()

    data_psf = np.nan_to_num(data, nan = 1**-7)

    ## FIXED CENTROIDS ##

    epsf.x_0.fixed = True
    epsf.y_0.fixed = True
    pos = Table(names = ['x_0', 'y_0'], data = [psf_sources['x_0'], psf_sources['y_0']])
    
    ## PERFORMING PSF PHOTOMETRY ##

    PSF_photometry = IterativelySubtractedPSFPhotometry(finder = psf_daofind,
                                                        group_maker = daogroup,
                                                        bkg_estimator = bkg_estimator,
                                                        psf_model = epsf,
                                                        fitter = fitter,
                                                        niters = 3,
                                                        aperture_radius = 6.,
                                                        fitshape = (11, 11))

    result_phot = PSF_photometry(image = data_psf, init_guesses = pos)
    residual_image = PSF_photometry.get_residual_image()
    
    #hdu = fits.PrimaryHDU(residual_image)
    #hdul = fits.HDUList([hdu])
    #hdul.writeto('residual_image_08_fixed_centroids.fits')
    print('Number of PSF stars found and analysed = {0}\n'.format(len(result_phot)))
    
    if do_plot == True:
        ## VISUALISE PSF IMAGE AND RESIDUALS ##
        plt.subplot(1, 2, 1)
        plt.imshow(data_psf, cmap = 'viridis', norm = LogNorm(), interpolation = 'nearest', origin = 'lower', vmin = 0.000001, vmax = 10**6)
        plt.title('input data')
        plt.colorbar(orientation = 'horizontal')

        plt.subplot(1, 2, 2)
        plt.imshow(residual_image, cmap = 'viridis', norm = LogNorm(), interpolation = 'nearest', origin = 'lower', vmin = 0.000001, vmax = 10**6)
        plt.title('residual image')
        plt.colorbar(orientation = 'horizontal')
        plt.gcf().set_size_inches(20, 14)
        plt.show()
        plt.close()
    elif do_plot == False:
        pass
    
    phot = result_phot   # REDEFINE PHOTOMETRY TABLE FOR EASE
    
    ## PHOTOMETRY: UNCERTAINTIES ##
    
    PSF_err = phot['flux_unc']
    PSF_flux = phot['flux_fit'] 
    
    ## APPARENT MAGNITUDES ##
    
    phot['apparent_mag'] = float('NaN')
    for i in range(0, len(phot)):
        # APPLY ARRAY-LOC DEP CORRECTION
        loc_corr = corr_data[int(phot['y_fit'][i])][int(phot['x_fit'][i])]
        if phot['flux_fit'][i] >= 0:
            phot['apparent_mag'][i] = zmag - 2.5 * math.log10(phot['flux_fit'][i] * aper_corr * loc_corr / counts)
    
    ## APPARENT MAGNITUDE: UNCERTAINTIES ##
    
    phot['apparent_mag_err'] = float('Nan')
    for i in range(0, len(phot)):
        if phot['flux_fit'][i] >= 0:
            phot['apparent_mag_err'][i] = pow(zmag_err**2 + (2.5*(pow((PSF_err[i] / PSF_flux[i])**2 + (ap_err / aper_corr)**2, 0.5) / (np.log(10)))**2), 0.5)
    
    ## EXPORT PHOTOMETRY FILE AND PRINT TO SCREEN ##
    phot['id', 'x_0', 'y_0', 'apparent_mag', 'apparent_mag_err'].write(
        r'C:\Users\lukeb\Documents\MPhys_Project_RRLs\Luke_RRLs_project\output_files\PAL5_PSFphot_01_epoch{0}_channel{1}.txt'.format(epoch, channel), format = 'csv', overwrite = False)
    
    print(phot['id', 'x_0', 'y_0', 'apparent_mag', 'apparent_mag_err'])
    print('\n\n')

In [ ]:
## MATCH STARS BETWEEN EPOCHS ##

file_1 = r'C:/Users/lukeb/Documents/MPhys_Project_RRLs/Luke_RRLs_project/output_files/PAL5_PSFphot_01_epoch1_channel3p6um.txt'

## EPOCH LOOP COUNTER ##

epoch = 1

time_list = [0]

for file_2 in glob.glob(r'C:/Users/lukeb/Documents/MPhys_Project_RRLs/Luke_RRLs_project/output_files/PAL5_PSFphot_01_epoch[2-9]_channel'+channel+'.txt', recursive = True) + glob.glob(r'C:/Users/lukeb/Documents/MPhys_Project_RRLs/Luke_RRLs_project/output_files/PAL5_PSFphot_01_epoch[0-9][0-9]_channel'+channel+'.txt', recursive = True):
    
    epoch += 1
    print('EPOCH NUMBER = {0}\n'.format(epoch))
    
    image_1 = 'PAL5_data/PAL5__e1/PAL5__e1_3p6um.fits'
    image_2 = str('PAL5_data/PAL5__e{0}/PAL5__e{0}_3p6um.fits'.format(epoch))
    
    with fits.open(image_1) as hdr_1_list:
        hdr_1 = hdr_1_list[0].header
        time1 = Time(hdr_1['DATE_OBS'])
        
    with fits.open(image_2) as hdr_2_list:
        hdr_2 = hdr_2_list[0].header
        time2 = Time(hdr_2['DATE_OBS'])
    
    delta_time = time2 - time1
    time_list.append(delta_time.sec)
    
    f1 = ascii.read(file_1, delimiter = ',')
    f2 = ascii.read(file_2, delimiter = ',')
    
    x_1 = f1['x_0']
    y_1 = f1['y_0']
    x_2 = f2['x_0']
    y_2 = f2['y_0']
    
    w1 = wcs.WCS(hdr_1)
    w2 = wcs.WCS(hdr_2)

    coord_1 = np.transpose((x_1, y_1))
    coord_2 = np.transpose((x_2, y_2))
    world_1 = w1.wcs_pix2world(coord_1, 0)
    world_2 = w2.wcs_pix2world(coord_2, 0)

    ra_1, dec_1 = world_1[:, 0], world_1[:, 1]
    ra_2, dec_2 = world_2[:, 0], world_2[:, 1]

    c_1 = SkyCoord(ra_1, dec_1, frame = 'icrs', unit = 'deg')
    c_2 = SkyCoord(ra_2, dec_2, frame = 'icrs', unit = 'deg')

    idx, d2d, d3d = c_2.match_to_catalog_sky(c_1)

    # APPEND RIGHT ASCENSION AND DECLINATION TO EXISTING FILES
    f1['ra'] = ra_1
    f1['dec'] = dec_1
    f2['ra'] = ra_2
    f2['dec'] = dec_2
    
    f1['ra_hms'] = str('null')
    f1['dec_dms'] = str('null')
    f2['ra_hms'] = str('null')
    f2['dec_dms'] = str('null')
    
    # FOLLOWING TUTORIAL, ENSURE MATCHES ARE SIGNIFICANT
    radius = 0.0001
    selection = (d2d > radius*u.deg)
    match_index = idx
    match_index[selection] = -99.
    matches = (match_index >= 0)

    mag_1 = f1['apparent_mag'][match_index][matches]
    mag_2 = f2['apparent_mag'][matches]
    mag_1_err = f1['apparent_mag_err'][match_index][matches]
    mag_2_err = f2['apparent_mag_err'][matches]
    delta_mag = mag_1 - mag_2
    print('Number of matched stars between epoch 1 and {0} = {1}\n'.format(epoch, len(delta_mag)))
    
    # CONSTRUCT TABLE OF MATCHED STARS
    matched_tbl_1 = Table()
    matched_tbl_2 = Table()
    ra_1m, dec_1m = f1['ra'][match_index][matches], f1['dec'][match_index][matches]
    ra_2m, dec_2m = f2['ra'][matches], f2['dec'][matches]
    matched_tbl_1['ra1'], matched_tbl_1['dec1'] = ra_1m, dec_1m
    matched_tbl_2['ra2'], matched_tbl_2['dec2'] = ra_2m, dec_2m
    matched_tbl_1['mag_1'] = mag_1
    matched_tbl_2['mag_2'] = mag_2
    matched_tbl_1['mag_1_err'] = mag_1_err
    matched_tbl_2['mag_2_err'] = mag_2_err
    
    # INITIALIZE COLUMNS FOR CONVERTING RIGHT ASCENSION AND DECLINATION FOR COMPARISON TO STAR CATALOGUE
    # RIGHT ASCENSION TO BE CONVERTED INTO HOURS:MINUTES:SECONDS
    # DECLINATION TO BE CONVERTED INTO DEGRESS:MINUTES:SECONDS
    matched_tbl_1['ra_hms'] = str('null')
    matched_tbl_1['dec_dms'] = str('null')
    matched_tbl_2['ra_hms'] = str('null')
    matched_tbl_2['dec_dms'] = str('null')
    
    # CONVERTING STRING COLUMNS IN TABLES INTO OBJECT
    for col in matched_tbl_1.itercols():
        if col.dtype.kind in 'SU':
            matched_tbl_1.replace_column(col.name, col.astype('object'))
    for col in matched_tbl_2.itercols():
        if col.dtype.kind in 'SU':
            matched_tbl_2.replace_column(col.name, col.astype('object'))
    
    # PERFORMING CONVERSIONS
    for i in range(len(matched_tbl_1)):
        ra_deg = Angle(matched_tbl_1['ra1'][i], u.deg)
        matched_tbl_1['ra_hms'][i] = ra_deg.to_string(unit = u.hour, sep=':')
    for i in range(len(matched_tbl_1)):
        dec_deg = Angle(matched_tbl_1['dec1'][i], u.deg)
        matched_tbl_1['dec_dms'][i] = dec_deg.to_string(unit = u.degree, sep=':')
    for i in range(len(matched_tbl_2)):
        ra_deg = Angle(matched_tbl_2['ra2'][i], u.deg)
        matched_tbl_2['ra_hms'][i] = ra_deg.to_string(unit = u.hour, sep=':')
    for i in range(len(matched_tbl_2)):
        dec_deg = Angle(matched_tbl_2['dec2'][i], u.deg)
        matched_tbl_2['dec_dms'][i] = dec_deg.to_string(unit = u.degree, sep=':')
    
    # EXPORT CONVERTED TABLES
    matched_tbl_1['ra_hms', 'dec_dms', 'mag_1', 'mag_1_err'].write(r'C:\Users\lukeb\Documents\MPhys_Project_RRLs\Luke_RRLs_project\output_files\PAL5_matched_stars_01_epoch1_{0}.txt'.format(channel), format = 'csv', overwrite = True)
    matched_tbl_2['ra_hms', 'dec_dms', 'mag_2', 'mag_2_err'].write(r'C:\Users\lukeb\Documents\MPhys_Project_RRLs\Luke_RRLs_project\output_files\PAL5_matched_stars_01_epoch{0}_{1}.txt'.format(epoch, channel), format = 'csv', overwrite = True)

In [ ]:
magnitudes = [16.5776,
              16.4341,
              16.4177,
              16.4532,
              16.4204,
              16.4732,
              16.4265,
              16.4563,
              16.4105,
              16.4760,
              16.4761,
              16.3585]

magnitudes_err = [0.03218,
                  0.03232,
                  0.03095,
                  0.03190,
                  0.02898,
                  0.03406,
                  0.03141,
                  0.03262,
                  0.03316,
                  0.03251,
                  0.03097,
                  0.03050]



#    elinewidth=2,
#    markeredgewidth=2
    
plt.plot(time_list, magnitudes, color = 'kx', linestyle='None', markersize = 5)
plt.errorbar(x = time_list, y = magnitudes, xerr = None, yerr = magnitudes_err, fmt = 'o', capsize = 5)
plt.title('RRL ID1')
plt.xlabel('time [seconds]')
plt.ylabel('apparent magnitude')
plt.grid()
plt.gca().invert_yaxis()
plt.gcf().set_size_inches(10, 5)
plt.show()
plt.close()